In [10]:
import numpy as np
import threading
import time
import decimal
from decimal import Decimal
from Channel import Channel

In [11]:
class NotInAlphabetException (Exception):
    pass
class NotABitException (Exception):
    pass

In [12]:
class LaplaceEncoder:
    
    def __init__(self, alphabet, channel):
        self.alphabet = alphabet
        self.channel = channel
        self.message = ''
        self.occurences = np.zeros(len(alphabet))
        self.predictions = []
        for index,letter in enumerate(alphabet):
            start = Decimal(int(np.sum(self.occurences[:index]+1)))/Decimal(int(np.sum(self.occurences+1)))
            end = Decimal(int(np.sum(self.occurences[:index+1]+1)))/Decimal(int(np.sum(self.occurences+1)))
            self.predictions.append([letter,start,end])
        decimal.getcontext().prec=100
        
    def get_bits(self, bit):
        channel.send_bits(bit)
        self.message += bit
    
    def encode(self, message):
        message = message
        range_defined = np.array([Decimal(0),Decimal(1)])
        
        for character in message:
            range_to_pick = self.predictions[np.argwhere(self.alphabet==character)[0][0]]
            range_to_pick = np.array([range_to_pick[1], range_to_pick[2]])
            while(True):
                middle = range_defined[0] + (range_defined[1]-range_defined[0])/2
                if range_to_pick[0] >= middle:
                    range_defined[0] = middle
                    self.get_bits('1')
                    decimal.getcontext().prec += 1
                elif range_to_pick[1] < middle:
                    range_defined[1] = middle
                    self.get_bits('0')
                else:
                    break
            self.predictions = []
            self.occurences[np.argwhere(self.alphabet==character)] += 1
            # new character
            for index,letter in enumerate(alphabet):
                range_length = range_to_pick[1]-range_to_pick[0]
                start = range_to_pick[0] + range_length * (Decimal(int(np.sum(self.occurences[:index]+1)))/Decimal(int(np.sum(self.occurences+1))))
                end = range_to_pick[0] + range_length * (Decimal(int(np.sum(self.occurences[:index+1]+1)))/Decimal(int(np.sum(self.occurences+1))))
                self.predictions.append([letter,start,end])
                
        # end: in general, range still not identified
        while (True):
            if range_defined[1] <= range_to_pick[1] and range_defined[0] >= range_to_pick[0]:
                break
            if range_defined[0] < range_to_pick[0]:
                self.get_bits('1')
                range_defined[0] = range_defined[0] + (range_defined[1]-range_defined[0])/2
            elif range_defined[1] > range_to_pick[1]:
                self.get_bits('0')
                range_defined[1] = range_defined[0] + (range_defined[1]-range_defined[0])/2
                
        closed = False
        while not closed:
            closed = self.channel.close_channel()

In [13]:
class LaplaceDecoder:
    def __init__(self, alphabet, verbose=False):
        self.alphabet = alphabet
        self.message_received = ''
        self.message_decoded = ''
        self.occurences = np.zeros(len(alphabet))
        self.predictions = []
        for index,letter in enumerate(alphabet):
            start = Decimal(int(np.sum(self.occurences[:index]+1)))/Decimal(int(np.sum(self.occurences+1)))
            end = Decimal(int(np.sum(self.occurences[:index+1]+1)))/Decimal(int(np.sum(self.occurences+1)))
            self.predictions.append(np.array([letter,start,end]))
        self.predictions = np.array(self.predictions)
        self.range_defined = np.array([Decimal(0),Decimal(1)])
        self.active = True
        self.verbose = verbose
        decimal.getcontext().prec=100
            
    def send (self, bit):
        
        if not self.active:
            return
        
        self.message_received = self.message_received + bit
        if self.verbose:
            print('Received '+bit)

        mittel = self.range_defined[0] + (self.range_defined[1]-self.range_defined[0])/2
    
        if bit=='0':
            self.predictions = self.predictions[np.where(self.predictions[:,1]<mittel)]
            self.range_defined[1] = mittel
        elif bit=='1':
            self.predictions = self.predictions[np.where(self.predictions[:,2]>=mittel)]
            self.range_defined[0] = mittel
        else:
            raise NotABitException
        
        # check if letters can be decoded
        while (True):
            # if only one letter remains, then clearly it can be decoded
            if len(self.predictions)==1:
                self.message_decoded += self.predictions[0][0]
                letter_range = self.predictions[0][1:]
                if self.verbose:
                    print('Decoded ' + self.predictions[0][0])
                self.occurences[np.argwhere(self.alphabet==self.predictions[0][0])] += 1
                if self.predictions[0][0]==alphabet[-1]:
                    self.active = False
                    return
                
                # add predictions
                self.predictions = []
                for index,letter in enumerate(alphabet):
                    range_length = letter_range[1]-letter_range[0]
                    start = letter_range[0] + range_length * (Decimal(int(np.sum(self.occurences[:index]+1)))/Decimal(int(np.sum(self.occurences+1))))
                    end = letter_range[0] + range_length * (Decimal(int(np.sum(self.occurences[:index+1]+1)))/Decimal(int(np.sum(self.occurences+1))))
                    self.predictions.append(np.array([letter,start,end]))
                self.predictions = np.array(self.predictions)
                self.predictions = self.predictions[np.where(self.predictions[:,1]<=self.range_defined[1])]
                self.predictions = self.predictions[np.where(self.predictions[:,2]>=self.range_defined[0])]
            else:
                break
        

In [14]:
import threading
import random
import math

random_message = random.choices(['a', 'b'], weights=[0.95, 0.05], k=300)
alphabet = np.array(list(set(random_message)) + ['END'])
message = random_message + ['END']
# create Channel
channel = Channel(latency=0.01)
encoder = LaplaceEncoder(alphabet, channel)
decoder = LaplaceDecoder(alphabet, verbose=False)
channel.attach_decoder(decoder)
channel_thread = threading.Thread(target=channel.open_channel, args=())
encoder_thread = threading.Thread(target=encoder.encode, args=(message,))
channel_thread.start()
encoder_thread.start()
channel_thread.join()
encoder_thread.join()
print('done')
print(decoder.message_decoded)
print(''.join(message))
print('Worked: ' + str(decoder.message_decoded==''.join(message)))
print(f'Encoding: {decoder.message_received}, so {100 * len(decoder.message_received)/(len(decoder.message_decoded)*math.log(len(alphabet),2))}% of the original message')

done
aaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaababaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaabaaaaaaaaabEND
aaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaababaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabaaaaaaaabaaaaaaaaabEND
Worked: True
Encoding: 000000001011101101110110110110001000111000100010011110110100011111000001010, so 15.61707310820439% of the original message
